# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 12
max_magic = 14

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

import os

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ['QRACK_MAX_CPU_QB']='-1'

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [3]:
import math
import random

sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.s(q)
    return 1

def x_to_z(circ, q):
    circ.h(q)
    return 1

def y_to_z(circ, q):
    circ.adjs(q)
    circ.h(q)
    return 2

def y_to_x(circ, q):
    circ.adjs(q)
    return 1

def z_to_x(circ, q):
    circ.h(q)
    return 1

def z_to_y(circ, q):
    circ.h(q)
    circ.s(q)
    return 2

def cx(circ, q1, q2):
    circ.mcx([q1], q2)
    return 1

def cy(circ, q1, q2):
    circ.mcy([q1], q2)
    return 1

def cz(circ, q1, q2):
    circ.mcz([q1], q2)
    return 1

def acx(circ, q1, q2):
    circ.macx([q1], q2)
    return 1

def acy(circ, q1, q2):
    circ.macy([q1], q2)
    return 1

def acz(circ, q1, q2):
    circ.macz([q1], q2)
    return 1

def swap(circ, q1, q2):
    circ.swap(q1, q2)
    return 1

def nswap(circ, q1, q2):
    circ.mcz([q1], q2)
    circ.swap(q1, q2)
    circ.mcz([q1], q2)
    return 3

def pswap(circ, q1, q2):
    circ.mcz([q1], q2)
    circ.swap(q1, q2)
    return 2

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.mcz([q1], q2)
    return 2

def iswap(circ, q1, q2):
    circ.iswap(q1, q2)
    return 1

def iiswap(circ, q1, q2):
    circ.adjiswap(q1, q2)
    return 1

def random_circuit(width, circ):
    t_count = 0
    gate_count = 0
    bit_depths = width * [0]

    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width
    
    for i in range(3 * width):
        # Single bit gates
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)
            
            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            g_count = gate(circ, j)
            gate_count += g_count
            bit_depths[j] += g_count

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3
                
            # Rotate around local Z axis
            rnd = random.randint(0, 3)
            if rnd == 0:
                circ.s(j)
            elif rnd == 1:
                circ.z(j)
            elif rnd == 2:
                circ.adjs(j)
            # else - identity
            if rnd < 3:
                gate_count += 1
                bit_depths[j] += 1
            
            if (t_count < max_magic) and (width * width * random.random() / max_magic) < 1:
                circ.u(j, 0, random.uniform(0, 4 * math.pi), 0)
                gate_count += 1
                bit_depths[j] += 1
                t_count += 1
            
        # Nearest-neighbor couplers:
        ############################
        # gate = gateSequence.pop(0)
        # gateSequence.append(gate)
        # for row in range(1, row_len, 2):
        #     for col in range(row_len):
        #         temp_row = row
        #         temp_col = col
        #         temp_row = temp_row + (1 if (gate & 2) else -1);
        #         temp_col = temp_col + (1 if (gate & 1) else 0)
        #
        #         if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
        #             continue
        #
        #         b1 = row * row_len + col
        #         b2 = temp_row * row_len + temp_col
        #
        #         if (b1 >= width) or (b2 >= width):
        #             continue
        #
        #         g = random.choice(two_bit_gates)
        #         g_count = g(circ, b1, b2)
        #         gate_count += g_count
        #         bit_depths[b1] += g_count
        #         bit_depths[b2] += g_count

        # Fully-connected couplers:
        ###########################
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            g = random.choice(two_bit_gates)
            g_count = g(circ, b1, b2)
            gate_count += g_count
            bit_depths[b1] += g_count
            bit_depths[b2] += g_count

    print("Gate count (before optimization): ", gate_count)
    # This might not be right:
    # print("Depth of critical path (before optimization): ", max(bit_depths))

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [4]:
import torch
import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc
from pyqrack import QrackSimulator, QrackCircuit
from qiskit.compiler.transpiler import transpile

qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
# qsim.set_hardware_encoded(True)
random_circuit(width, qsim)
qsim.out_to_file('qrack_circuit.chp')
circ = QrackSimulator.file_to_qiskit_circuit('qrack_circuit.chp')

# basis_gates = ["rz", "h", "x", "y", "z", "s", "sdg", "sqrtx", "sqrtz", "cx", "cy", "cz", "swap", "iswap", ]
# circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

# circ.qasm(filename='qiskit_circuit.qasm')

# print(circ)

print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))

print("(Near-Clifford) depth of critical path: ", circ.depth())

2023-06-19 10:48:51.143723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Gate count (before optimization):  1205
(Near-Clifford) gate count:  376
(Near-Clifford) depth of critical path:  116


In [5]:
net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

print(net.sample(allow_state=True, batch=1024, format="count_dict_bin"))

print(circ)
# circ = net.to_qiskit()

{'0000000010010111101111110': 1, '0000000101001100101110010': 1, '0000000110100001110100010': 1, '0000000110110001110101010': 1, '0000001000100100010100110': 1, '0000001000101110001111011': 1, '0000001001110100001111111': 1, '0000001001111110010000000': 1, '0000001010100110011000000': 1, '0000001010111001001010000': 1, '0000001010111110011011001': 1, '0000001100110001001100001': 1, '0000001100111111001110011': 1, '0000001101010010101110001': 1, '0000001101010011111110000': 1, '0000001101011111000000010': 1, '0000001110011100100101100': 1, '0000001110101111010110101': 1, '0000001111000100111010011': 1, '0000010010001111110110110': 1, '0000010101010110000111011': 1, '0000010110000011001011010': 1, '0000010111001001100001011': 1, '0000011000001110011000011': 1, '0000011000011000011011001': 1, '0000011001010000111100100': 1, '0000011001101110000001001': 1, '0000011011100001110001111': 1, '0000011101000001010000100': 1, '0000011101100101100111010': 1, '0000011101101011100000101': 1, '000010

If the basis gates can be converted, a Qiskit `QuantumCircuit` can also be read into a `QrackCircuit`.